In [1]:
import fitz  # PyMuPDF
from pprint import pprint
import pandas as pd

doc = fitz.open("SampleData/sv600_c_normal.pdf")
#  doc = fitz.open("sv600_c_normal.pdf")

metadata = doc.metadata
pprint(metadata)
print("--------------------------------")

for page in doc:
    text = page.get_text()
    print(text)
    print("--------------------------------")


{'author': '',
 'creationDate': "D:20130826175213+09'00'",
 'creator': 'PFU ScanSnap Manager 6.2.10 #SV600',
 'encryption': None,
 'format': 'PDF 1.3',
 'keywords': '',
 'modDate': "D:20130826175213+09'00'",
 'producer': 'Adobe PDF Scan Library 3.2',
 'subject': '',
 'title': '',
 'trapped': ''}
--------------------------------

--------------------------------


In [2]:
def digital_pdf_get_text(pdf_path:str)->list[str]:
    """
    """
    doc = fitz.open(pdf_path)
    text = []
    for page in doc:
        text.append(page.get_text())
    return text

digital_pdf_get_text("SampleData/sample-report_(1).pdf")


['Sample Team\nPrepared By\nsample-files.com\nMulti-Page\nReport\n“A comprehensive and content-heavy report that\nincludes text, images, and tables for thorough\ntesting of pagination and complex layouts.”\n',
 'Table of Contents\n1. Introduction\n2. Market Analysis\n3. Data Analysis\n4. Product Overview\n5. Results & Discussion\n6. Marketing Strategy\n7. Sales Projections\n8. Launch Timeline\nThis sample PDF file is provided by Sample-Files.com. Visit us for more sample files and resource.\n',
 'Introduction\nThis section introduces the report and highlights the\nkey objectives. The purpose of this report is to analyze\ndata, evaluate outcomes, and provide insights for\nfuture decisions. This analysis is based on various data\nsources that include quantitative and qualitative\ninputs. Note: Add an image here illustrating the\nconcept of data analysis or research methodology. \nID\nMetric\nValue\nRemarks\n1\nMetric 1\n70\nValid Data\n2\nMetric 2\n431\nValid Data\n3\nMetric 3\n186\nVali

In [ ]:
def merge_imageText_with_pdfText(image_text:list[str],
                                  pdf_text:list[str])->list[str]:
    """
    """
    assert len(image_text) == len(pdf_text)
    merged_text = [image_text[i] + pdf_text[i] for i in range(len(image_text))]
    return merged_text

merge_imageText_with_pdfText(image_text, pdf_text)

In [ ]:
def create_page_finder_result_template(search_conditions):
    """
    Create a template DataFrame for storing page finding results.
    
    This function generates a standardized template structure for tracking
    the results of page finding operations across PDF documents. The template
    includes fields for document metadata, page information, and search condition
    results.
    
    Returns:
        pandas.DataFrame: A DataFrame with the following columns:
            - Index: Dictionary field for storing page indices
            - File_Name: String field for the name of the processed file
            - File_Path: String field for the full path to the file
            - Page_Count: Integer field for total number of pages in the document
            - Page_Number_Found: Integer field for the page number where search
              conditions were satisfied
            - SearchConditions_Satisfied: Dictionary field for storing boolean
              results of each search condition (e.g., {"A": True, "B": False})
    
    Example:
        >>> template_df = create_page_finder_result_template()
        >>> print(template_df.columns.tolist())
        ['Index', 'File_Name', 'File_Path', 'Page_Count', 'Page_Number_Found', 'SearchConditions_Satisfied']
    
    Note:
        - The DataFrame is initialized with empty/default values
        - SearchConditions_Satisfied field is designed to store results from
          search_conditions_document function
        - This template serves as a foundation for building result datasets
          from multiple document searches
    """
    page_finder_result_template:dict = {
        "Index":{},
        "File_Name": "",
        "File_Path": "",
        "Page_Count": 0,
        "Page_Number_Found": 0,
        **search_conditions,
        # "SearchConditions_Satisfied":{}, #{"A":True, "B":False},
    }
    df = pd.DataFrame(page_finder_result_template)
    return df

create_page_finder_result_template({"A":None, "B":None, "C":None})

In [ ]:
def search_conditions_document(
                        doc:list[str],
                        search_conditions:dict[str, callable],
                        ):
    """
    """
    search_conditions_output_result=[None for _ in doc]

    for i, page_text in enumerate(doc):            
        search_conditions_output={k:None for k in search_conditions}
        
        for condition_name, condition_func in search_conditions.items():
            if condition_func("", page_text):
                search_conditions_output[condition_name]=True
            else:
                search_conditions_output[condition_name]=False

        search_conditions_output_result[i] = search_conditions_output #Use update instead of append for speed
    
    return search_conditions_output_result

def page_find_from_conditions(search_conditions_output:dict)->int:
    #TODO: Implement logic
    return 4


search_conditions = {
    "A": lambda x, page_text: "sample".lower() in page_text.lower(),
    "B": lambda x, page_text: "results" in page_text,
    "C": lambda x, page_text: "results".upper() in page_text.upper(),
}
doc_texts = [page.get_text() for page in doc]
search_conditions_output = search_conditions_document(doc_texts, search_conditions)

pprint(search_conditions_output)
print(page_find_from_conditions(search_conditions_output))

In [3]:
import pytesseract
from PIL import Image
import io

def extract_text_from_image_ocr(image_path_or_bytes,
                                tesseract_path=None,
                                tesseract_config_mode="--psm 4"
                                ):
    """
    Extract text from an image using Tesseract OCR.
    
    Args:
        image_path_or_bytes: Either a file path (str) or image bytes
        tesseract_path: Optional path to tesseract executable (Windows users may need this)
    
    Returns:
        str: Extracted text from the image
        
    Raises:
        Exception: If OCR fails or Tesseract is not found
    """
    try:
        # Set tesseract path if provided (common on Windows)
        if tesseract_path:
            pytesseract.pytesseract.tesseract_cmd = tesseract_path
        
        # Handle both file paths and image bytes
        if isinstance(image_path_or_bytes, str):
            # File path provided
            image = Image.open(image_path_or_bytes)
        else:
            # Image bytes provided
            image = Image.open(io.BytesIO(image_path_or_bytes))
        
        # Extract text using OCR
        text = pytesseract.image_to_string(image,
                                           config=tesseract_config_mode)
        
        return text.strip()
        
    except Exception as e:
        raise Exception(f"OCR failed: {str(e)}")

def extract_text_from_pdf_images_ocr(doc,
                                     tesseract_path=None):
    """
    Extract text from all images in a PDF using OCR.
    
    Args:
        doc: PyMuPDF document object
        tesseract_path: Optional path to tesseract executable
    
    Returns:
        dict: Dictionary mapping page numbers to lists of extracted text from images
    """
    results = {}
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        images = page.get_images(full=False)
        
        if not images:
            continue
            
        page_results = []
        
        for img in images:
            xref = img[0]
            
            try:
                # Create pixmap from image
                pix = fitz.Pixmap(doc, xref)
                
                # Convert to PIL Image for OCR
                img_data = pix.tobytes("png")
                
                # Extract text using OCR
                extracted_text = extract_text_from_image_ocr(img_data, tesseract_path)
                
                if extracted_text:
                    page_results.append({
                        'xref': xref,
                        'text': extracted_text,
                        'size': (pix.width, pix.height)
                    })
                
                pix = None  # Free memory
                
            except Exception as e:
                print(f"Failed to process image {xref} on page {page_num + 1}: {e}")
                continue
        
        if page_results:
            # results[page_num] = page_results
            results[page_num] = page_results['text']
    
    return results

# Alternative function using easyocr (no Tesseract installation needed)
def extract_text_from_image_easyocr(image_path_or_bytes):
    """
    Extract text from an image using EasyOCR (no external dependencies).
    
    Args:
        image_path_or_bytes: Either a file path (str) or image bytes
    
    Returns:
        str: Extracted text from the image
    """
    try:
        import easyocr
        
        # Initialize reader (first time will download models)
        reader = easyocr.Reader(['en'])
        
        # Handle both file paths and image bytes
        if isinstance(image_path_or_bytes, str):
            # File path provided
            result = reader.readtext(image_path_or_bytes)
        else:
            # Image bytes provided - save temporarily
            temp_img = Image.open(io.BytesIO(image_path_or_bytes))
            temp_path = "temp_image.png"
            temp_img.save(temp_path)
            result = reader.readtext(temp_path)
            import os
            os.remove(temp_path)  # Clean up
        
        # Extract text from results
        text = ' '.join([item[1] for item in result])
        return text.strip()
        
    except Exception as e:
        raise Exception(f"EasyOCR failed: {str(e)}")

In [5]:
%%time
tesseract_path = r"C:\Users\mmumbaiwala\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
# Example 1: OCR on a specific image
image_text = extract_text_from_image_ocr("image_5.png",
                                         tesseract_path=tesseract_path,
                                         tesseract_config_mode="--psm 1")

image_text

CPU times: total: 93.8 ms
Wall time: 4.21 s


'isiness report\n\nNew customer\'s development\nand increasing the sale of product\n\nMy country economy at this season keeps escaping from Odoba of business though holds a crude oil\nhigh so on unstable element that continues still, and recovering gradually and well.\nIn the IT industry, there is an influence such as competing intensification in narrowing investment field.\n\n[The main product and service at this season]\n\nFrom the product headquarters\n\nIn the image business, the new model turning on of the A3 high-speed,\ntwo sided color scanner that achieved a high-speed reading aimed at.\nwroom was established in United States, Europe, and Asia/Oceania.\n\n@image business\n\n1) Scanner class\n\nA3 high-speed, two sided color scanner "fi-5900C" that 100 high-n\nfunction to enable industry-leading was installed was announced in\nScanSnap gotten popular because of an office and individual use.\n\n2) DLM solution scanner\n\nThe DLM solution that used received the rise of the concern

In [6]:
doc = fitz.open("SampleData/sv600_c_normal.pdf")
extract_text_from_pdf_images_ocr(doc=doc,
                                 tesseract_path=tesseract_path)

TypeError: list indices must be integers or slices, not str

In [ ]:
print(image_text)

In [ ]:


# Example 2: OCR on all images in your PDF
# For Windows users who installed Tesseract to default location:

ocr_results = extract_text_from_pdf_images_ocr(doc, tesseract_path)

# Example 3: Using EasyOCR (no Tesseract needed)
# easyocr_text = extract_text_from_image_easyocr("image_5.png")

# Print results
for page_num, images in ocr_results.items():
    print(f"\nPage {page_num + 1}:")
    for img_info in images:
        print(f"  Image {img_info['xref']}: {img_info['text'][:100]}...")

In [ ]:
# from attrs import define, field, validators
# from typing import Optional, Tuple

# @define
# class StaticTextElement:
#     search_text: str = field(validator=validators.instance_of(str))
#     max_errors: int = field(validator=validators.instance_of(int))
#     max_error_rate: float = field(validator=validators.instance_of(float))
#     match_case: bool = field(validator=validators.instance_of(bool))
#     # region: Optional[Tuple[int, int, int, int]] = None  # (x1, y1, x2, y2) bounding box
#     # use_regex: bool = False  # Support regex search

# @define
# class MatchResult:
#     search_text: str = field(validator=validators.instance_of(str))
#     errors: int = field(validator=validators.instance_of(int))
#     error_rate: float = field(validator=validators.instance_of(float))
#     match_case: bool = field(validator=validators.instance_of(bool))
#     success: bool = field(validator=validators.instance_of(bool))
#     # matched_region: Optional[Tuple[int, int, int, int]] = None
#     # confidence: Optional[float] = None  # Confidence score from OCR or matching algorithm


In [ ]:
# StaticTextElement(search_text="sample",
#                   max_errors=1,
#                   max_error_rate=0.5,
#                   match_case=True
#                   )


StaticTextElement(search_text='sample', max_errors=1, max_error_rate=0.5, match_case=True)

In [ ]:

# from rapidfuzz import fuzz, distance
# from attrs import define, field, validators
# from typing import Optional, Tuple

# @define
# class StaticTextElement:
#     search_text: str = field(validator=validators.instance_of(str))
#     max_errors: int = field(validator=validators.instance_of(int))
#     max_error_rate: float = field(validator=validators.instance_of(float))
#     match_case: bool = field(validator=validators.instance_of(bool))

# @define
# class MatchResult:
#     search_text: str = field(validator=validators.instance_of(str))
#     errors: int = field(validator=validators.instance_of(int))
#     error_rate: float = field(validator=validators.instance_of(float))
#     match_case: bool = field(validator=validators.instance_of(bool))
#     success: bool = field(validator=validators.instance_of(bool))

# def search_static_text_elements(elements: list[StaticTextElement],
#                                 text: str,
#                                 max_hypothesis: int = 3,
#                                 max_window_size: int = 11,
#                                 debug_mode: bool = False) -> list[MatchResult]:
#     """
#     Search for static text elements with fuzzy matching.
    
#     Args:
#         elements: List of StaticTextElement to search for
#         text: Text to search within
#         max_hypothesis: Maximum number of best matches to keep for each element
#         max_window_size: Additional characters to add to pattern length for window size
#         debug_mode: If True, print detailed search process. If False, print clean output.
    
#     Returns:
#         List of MatchResult objects (one per element, with best match)
#     """
#     results = []

#     for element in elements:
#         if element.match_case:
#             text_to_search = text
#             pattern = element.search_text
#         else:
#             text_to_search = text.lower()
#             pattern = element.search_text.lower()

#         pattern_len = len(pattern)
#         window_size = pattern_len + max_window_size  # allow some flexibility
        
#         # Keep track of top N hypotheses for this element
#         hypotheses = []  # List of (score, errors, substring) tuples
        
#         if debug_mode:
#             print(f"\n=== Searching for '{element.search_text}' ===")
#             print(f"Pattern (normalized): '{pattern}' (length: {pattern_len})")
#             print(f"Window size: {window_size}")
#             print(f"Max hypotheses to keep: {max_hypothesis}")

#         # Quick exact match shortcut
#         if pattern in text_to_search:
#             if debug_mode:
#                 print(f"✓ Exact match found!")
#             results.append(MatchResult(
#                 search_text=element.search_text,
#                 errors=0,
#                 error_rate=0.0,
#                 match_case=element.match_case,
#                 success=True
#             ))
#             continue

#         # Sliding window fuzzy search
#         for i in range(len(text_to_search) - window_size + 1):
#             snippet = text_to_search[i:i+window_size]
            
#             # Use partial_ratio for scoring
#             score = fuzz.partial_ratio(pattern, snippet)
            
#             # Find the best substring within the snippet - try different lengths
#             best_substring_in_snippet = None
#             best_substring_score = -1
            
#             # Try different substring lengths around the pattern length
#             # This allows us to find complete words that might be longer/shorter than pattern
#             for length_offset in range(-2, 4):  # Try lengths: pattern_len-2, pattern_len-1, pattern_len, pattern_len+1, pattern_len+2
#                 target_length = pattern_len + length_offset
#                 if target_length <= 0 or target_length > len(snippet):
#                     continue
                    
#                 for j in range(len(snippet) - target_length + 1):
#                     sub_snippet = snippet[j:j + target_length]
#                     sub_score = fuzz.ratio(pattern, sub_snippet)
                    
#                     if sub_score > best_substring_score:
#                         best_substring_score = sub_score
#                         best_substring_in_snippet = sub_snippet
            
#             # Calculate Levenshtein distance for the best substring
#             if best_substring_in_snippet:
#                 errors = distance.Levenshtein.distance(pattern, best_substring_in_snippet)
                
#                 # Create hypothesis tuple: (score, errors, substring, position)
#                 hypothesis = (score, errors, best_substring_in_snippet, i)
                
#                 # Check if this hypothesis is worth keeping
#                 should_add = False
                
#                 if len(hypotheses) < max_hypothesis:
#                     # Always add if we haven't reached max_hypothesis
#                     should_add = True
#                 else:
#                     # Check if this hypothesis is better than any existing ones
#                     worst_hypothesis = max(hypotheses, key=lambda x: (x[1], -x[0]))  # Worst by errors, then by score
#                     if errors < worst_hypothesis[1] or (errors == worst_hypothesis[1] and score > worst_hypothesis[0]):
#                         should_add = True
                
#                 if should_add:
#                     # Add new hypothesis
#                     hypotheses.append(hypothesis)
                    
#                     # Keep only top max_hypothesis hypotheses
#                     if len(hypotheses) > max_hypothesis:
#                         # Sort by errors (ascending), then by score (descending)
#                         hypotheses.sort(key=lambda x: (x[1], -x[0]))
#                         hypotheses = hypotheses[:max_hypothesis]
                    
#                     if debug_mode:
#                         print(f"  New hypothesis added: score={score:.1f}, errors={errors}, substring='{best_substring_in_snippet}', pos={i}")
#                         print(f"  Current top {len(hypotheses)} hypotheses:")
#                         for idx, (s, e, sub, pos) in enumerate(hypotheses):
#                             print(f"    {idx+1}. Score: {s:.1f}, Errors: {e}, Substring: '{sub}', Position: {pos}")

#         # Select the best hypothesis (lowest errors, then highest score)
#         if hypotheses:
#             # Sort by errors (ascending), then by score (descending)
#             hypotheses.sort(key=lambda x: (x[1], -x[0]))
#             best_score, best_errors, best_substring, best_position = hypotheses[0]
            
#             if debug_mode:
#                 print(f"\n  Best hypothesis selected:")
#                 print(f"    Score: {best_score:.1f}, Errors: {best_errors}, Substring: '{best_substring}', Position: {best_position}")
            
#             # Calculate error rate based on actual substring length
#             actual_length = len(best_substring) if best_substring else pattern_len
#             error_rate = best_errors / max(1, actual_length)
            
#             success = (
#                 best_errors <= element.max_errors and
#                 error_rate <= element.max_error_rate
#             )
            
#             if debug_mode:
#                 print(f"  Final result: errors={best_errors}, error_rate={error_rate:.4f}, success={success}")
            
#             results.append(MatchResult(
#                 search_text=element.search_text,
#                 errors=best_errors,
#                 error_rate=error_rate,
#                 match_case=element.match_case,
#                 success=success
#             ))
#         else:
#             if debug_mode:
#                 print(f"  No hypotheses found!")
#             results.append(MatchResult(
#                 search_text=element.search_text,
#                 errors=-1,
#                 error_rate=1.0,
#                 match_case=element.match_case,
#                 success=False
#             ))

#     return results
# # Example usage:
# if __name__ == "__main__":
#     elements = [
#         StaticTextElement(search_text="SuperSpecial_secret",
#                           max_errors=3,
#                           max_error_rate=0.2,
#                           match_case=False
#                           ),
#         StaticTextElement(search_text="AnotherText",
#                           max_errors=3,
#                           max_error_rate=0.5,
#                           match_case=False
#                           ),
#         StaticTextElement(search_text="book",
#                           max_errors=3,
#                           max_error_rate=0.1,
#                           match_case=False
#                           )
#     ]

#     ocr_text = "Some big noisy OCR output that includes Superspecial_secret and other boak AnnotherText ..."
#     matches = search_static_text_elements(elements, ocr_text)

#     for match in matches:
#         print(match)



MatchResult(search_text='SuperSpecial_secret', errors=0, error_rate=0.0, match_case=False, success=True)
MatchResult(search_text='AnotherText', errors=1, error_rate=0.08333333333333333, match_case=False, success=True)
MatchResult(search_text='book', errors=1, error_rate=0.25, match_case=False, success=False)
